In [1]:
import pandas as pd, numpy as np, os
import pickle
from tqdm import tqdm

In [2]:
get_baseline_periHD_df = pd.read_csv('./dataset/rawdata/PeirHD_ExternalData_RawData-Sample-500.csv')
get_baseline_periHD_df['洗腎紀錄時間'] = pd.to_datetime(get_baseline_periHD_df['洗腎紀錄時間'])
get_total_intraHD_df = pd.read_csv('./dataset/rawdata/intraHD2016_20_en.csv')
get_total_intraHD_df['洗腎紀錄時間'] = pd.to_datetime(get_total_intraHD_df['洗腎紀錄時間'])

/tmp/ipykernel_44062/2733680076.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  get_total_intraHD_df = pd.read_csv('./dataset/rawdata/intraHD2016_20_en.csv')


In [3]:
save_path = '/ssd8/chih/project/yadong/predict_baseline_version02_ACM/dataset/baseline-testdataset_pickle_save'
save_folder = 'sample_500_seed_42'
os.makedirs(os.path.join(save_path, save_folder), exist_ok=True)
total_intraHD_data_count = 0
total_periHD_not_effect_data_count = 0
for next_iloc in tqdm(range(len(get_baseline_periHD_df))):
    get_single_periHD_df = get_baseline_periHD_df.iloc[next_iloc]
    get_single_periHD_ID = get_single_periHD_df['ID']
    get_single_periHD_time = get_single_periHD_df['洗腎紀錄時間']
    get_periHD_match_intraHD_ID_df = get_total_intraHD_df[get_total_intraHD_df['ID']==get_single_periHD_ID]
    get_periHD_match_intraHD_time_df = get_periHD_match_intraHD_ID_df[get_periHD_match_intraHD_ID_df['洗腎紀錄時間']==get_single_periHD_time]
    if get_periHD_match_intraHD_time_df.shape[0] !=0:
        get_periHD_match_intraHD_time_df_copy = get_periHD_match_intraHD_time_df.copy()
        get_periHD_match_intraHD_time_df_copy['sbp'] = get_periHD_match_intraHD_time_df_copy['sbp'].apply(lambda x: get_periHD_match_intraHD_time_df['sbp'].iloc[0] if (x>240) or (x<40) or type(x)!=int else x)
        get_periHD_match_intraHD_time_df_copy['dbp'] = get_periHD_match_intraHD_time_df_copy['dbp'].apply(lambda x: get_periHD_match_intraHD_time_df['dbp'].iloc[0] if (x>140) or (x<30) or type(x)!=int  else x)
        total_intraHD_data_count+=len(get_periHD_match_intraHD_time_df)
        # print(sum([0 if type(i)==int else 1 for i in get_periHD_match_intraHD_time_df_copy['sbp']]),sum([0 if type(i)==int else 1 for i in get_periHD_match_intraHD_time_df_copy['dbp']]))
        get_periHD_match_intraHD_time_df_copy['處置其他'] = get_periHD_match_intraHD_time_df_copy['處置其他'].fillna('無')
        get_periHD_match_intraHD_time_df_copy['症狀處置'] = get_periHD_match_intraHD_time_df_copy['症狀處置'].fillna('無')
        get_periHD_match_intraHD_time_df_copy['處置其他+症狀處置（描述）'] = get_periHD_match_intraHD_time_df_copy['處置其他']+' || '+ get_periHD_match_intraHD_time_df_copy['症狀處置']
        get_periHD_match_intraHD_time_df = get_periHD_match_intraHD_time_df_copy
        save_dict = {"ID":str(get_single_periHD_ID), "DateTime":get_single_periHD_time, "PeriHD": get_single_periHD_df, "IntraHD": get_periHD_match_intraHD_time_df}
        with open(os.path.join(save_path,save_folder, "baseline_01_sample_500_ID-{}-DateTime-{}.pkl".format(get_single_periHD_ID, str(get_single_periHD_time).replace(':','-').replace(' ','-'))), "wb") as f:
            save_dict = {"ID":get_single_periHD_ID, "DateTime":get_single_periHD_time, "PeriHD": get_single_periHD_df, "IntraHD": get_periHD_match_intraHD_time_df}
            pickle.dump(save_dict, f)


100%|██████████| 500/500 [01:06<00:00,  7.50it/s]
